In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
bus_data = pd.read_csv("../data/iett_otobus_duraklar.csv")

In [3]:
bus_data.head()

,geometry
0,POINT (29.2763499999941 40.8183030005252)
1,POINT (29.266183999994 40.8140130005245)
2,POINT (29.3429089999946 40.8837970005355)
3,POINT (29.3517150000085 40.8281299991206)
4,POINT (29.3586189999948 40.8285630005268)


In [4]:
from shapely import wkt

bus_data["geometry"] = bus_data["geometry"].apply(wkt.loads)

gdf_points = gpd.GeoDataFrame(bus_data, geometry="geometry", crs="EPSG:4326")

gdf_districts = gpd.read_file("../data/istanbul-districts.json")

gdf_points = gdf_points.to_crs(gdf_districts.crs)

result = gpd.sjoin(gdf_points, gdf_districts, how="left", predicate="within")

In [5]:
result.head()

,geometry,index_right,id,name
0,POINT (29.27635 40.8183),5.0,relation/1276010,Tuzla
1,POINT (29.26618 40.81401),5.0,relation/1276010,Tuzla
2,POINT (29.34291 40.8838),5.0,relation/1276010,Tuzla
3,POINT (29.35172 40.82813),5.0,relation/1276010,Tuzla
4,POINT (29.35862 40.82856),5.0,relation/1276010,Tuzla


In [6]:
code_dict = {
    "Adalar": 0,
    "Arnavutkoy": 1,
    "Atasehir": 2,
    "Avcilar": 3,
    "Bagcilar": 4,
    "Bahcelievler": 5,
    "Bakirkoy": 6,
    "Basaksehir": 7,
    "Bayrampasa": 8,
    "Besiktas": 9,
    "Beykoz": 10,
    "Beylikduzu": 11,
    "Beyoglu": 12,
    "Buyukcekmece": 13,
    "Catalca": 14,
    "Cekmekoy": 15,
    "Esenler": 16,
    "Esenyurt": 17,
    "Eyupsultan": 18,
    "Fatih": 19,
    "Gaziosmanpasa": 20,
    "Gungoren": 21,
    "Kadikoy": 22,
    "Kagithane": 23,
    "Kartal": 24,
    "Kucukcekmece": 25,
    "Maltepe": 26,
    "Pendik": 27,
    "Sancaktepe": 28,
    "Sariyer": 29,
    "Silivri": 30,
    "Sultanbeyli": 31,
    "Sultangazi": 32,
    "Sile": 33,
    "Sisli": 34,
    "Tuzla": 35,
    "Umraniye": 36,
    "Uskudar": 37,
    "Zeytinburnu": 38
}


In [7]:
result["name"] = result["name"].apply(lambda x: code_dict[x] if x in code_dict else None).astype("Int64")

result.dropna(subset=["name"], inplace=True)

In [8]:
result.drop(columns=["id", "index_right"], inplace=True)

In [9]:
result.to_csv("../data/bus_data_with_districts.csv", index=False)